In [9]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score

In [10]:
Train = pd.read_csv('dataTrain.csv')
Test = pd.read_csv('dataTest.csv')

X_train = Train.drop('output', axis=1)
y_train = Train['output']

X_test = Test.drop('output', axis=1)
y_test = Test['output']

maxs = X_train.max().tolist()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((241, 12), (61, 12), (241,), (61,))

In [11]:
def trainMLP(HLS):
    regr = MLPClassifier(hidden_layer_sizes=HLS,random_state=1, max_iter=5000)
    regr.fit(X_train, y_train)

    y_pred = regr.predict(X_test)

    print('')
    print('Architecture: {list}'.format(list = HLS))
    acc_score = accuracy_score(y_test, y_pred)
    print("Accuracy: {:.3f}".format(acc_score))
    rec_score = recall_score(y_test, y_pred)
    print("Recall: {:.3f}".format(rec_score))
    prec_score = precision_score(y_test, y_pred)
    print("Precision Score: {:.3f}".format(prec_score))
    F1_score = f1_score(y_test, y_pred)
    print("F1-Score: {:.3f}".format(F1_score))
    kappa = cohen_kappa_score(y_test, y_pred)
    print("Kappa Score: {:.3f}".format(kappa))

In [12]:
Architectures = [[5],[10], [5, 5], [5, 10],[10, 10]]

for i in Architectures: trainMLP(i)


Architecture: [5]
Accuracy: 0.869
Recall: 0.906
Precision Score: 0.853
F1-Score: 0.879
Kappa Score: 0.736

Architecture: [10]
Accuracy: 0.885
Recall: 0.875
Precision Score: 0.903
F1-Score: 0.889
Kappa Score: 0.770

Architecture: [5, 5]
Accuracy: 0.852
Recall: 0.812
Precision Score: 0.897
F1-Score: 0.852
Kappa Score: 0.706

Architecture: [5, 10]
Accuracy: 0.852
Recall: 0.844
Precision Score: 0.871
F1-Score: 0.857
Kappa Score: 0.705

Architecture: [10, 10]
Accuracy: 0.852
Recall: 0.781
Precision Score: 0.926
F1-Score: 0.847
Kappa Score: 0.707
